In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense, Conv2D, Flatten, MaxPooling2D, Dropout, LSTM, TimeDistributed
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

# --- HELPER FUNCTIONS ---

def create_lstm_dataset(X, y, time_steps=10):
    """Converts data into sequences for LSTM"""
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps)]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)
    
def pad_features(X, target_features):
    """Pads feature vector X to have target_features columns"""
    num_samples = X.shape[0]
    num_features = X.shape[1]
    pad_width = target_features - num_features
    if pad_width < 0:
        return X[:, :target_features]
    padding = np.zeros((num_samples, pad_width))
    return np.concatenate([X, padding], axis=1)

# --- 1. DATA PREPARATION (Load all data) ---
print("Loading data...")
train_data = pd.read_csv('modified_data_train.csv')
test_data  = pd.read_csv('modified_data_test.csv')

# --- Prep for Stage 1 (Binary) ---
train_data['binary_fault'] = train_data['faultNumber'].apply(lambda x: 0 if x == 0 else 1)
test_data['binary_fault'] = test_data['faultNumber'].apply(lambda x: 0 if x == 0 else 1)

y_train_s1 = train_data['binary_fault'].astype(int)
y_test_s1 = test_data['binary_fault'].astype(int)

# --- Prep for Stage 2 (Multiclass) ---
y_train_s2_labels = train_data['faultNumber'].astype(int)
y_test_s2_labels = test_data['faultNumber'].astype(int)

all_labels = pd.concat([y_train_s2_labels, y_test_s2_labels])
N_CLASSES = int(all_labels.max()) + 1
print(f"N_CLASSES set to {N_CLASSES}")

y_train_s2_cat = to_categorical(y_train_s2_labels, num_classes=N_CLASSES)
y_test_s2_cat = to_categorical(y_test_s2_labels, num_classes=N_CLASSES)

# --- Prep Features (X) ---
drop_cols = ['faultNumber', 'simulationRun', 'sample', 'binary_fault']
X_train = train_data.drop(columns=drop_cols, errors='ignore')
X_test = test_data.drop(columns=drop_cols, errors='ignore')

common_cols = sorted(list(set(X_train.columns).intersection(set(X_test.columns))))
X_train = X_train[common_cols]
X_test  = X_test[common_cols]

print(f"Data prepared with {len(common_cols)} common features.")

# --- 2. SCALING & PADDING (Shared) ---
print("Scaling and padding data...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

N_FEATURES = X_train_scaled.shape[1] # 37

# Pad features for 7x7 image
TARGET_DIM = int(np.ceil(np.sqrt(N_FEATURES)))
IMG_ROWS, IMG_COLS = TARGET_DIM, TARGET_DIM
TARGET_FEATURES = TARGET_DIM * TARGET_DIM
print(f"Padding from {N_FEATURES} to {TARGET_FEATURES} for ({IMG_ROWS}, {IMG_COLS}) image.")

X_train_padded = pad_features(X_train_scaled, TARGET_FEATURES)
X_test_padded = pad_features(X_test_scaled, TARGET_FEATURES)

# --- Define Model Parameters ---
TIME_STEPS = 10
HYBRID_INPUT_SHAPE = (TIME_STEPS, IMG_ROWS, IMG_COLS, 1)

# Define Early Stopping
early_stopper = EarlyStopping(
    monitor='val_loss', patience=3, verbose=1, restore_best_weights=True
)

# ==========================================================
#  STAGE 1: HYBRID CNN-LSTM (BINARY)
# ==========================================================

print("\n--- Preparing Data for Hybrid Stage 1 ---")
# Create sequences from the PADDED data
X_train_s1_seq, y_train_s1_seq = create_lstm_dataset(X_train_padded, y_train_s1.values, TIME_STEPS)
X_test_s1_seq, y_test_s1_seq = create_lstm_dataset(X_test_padded, y_test_s1.values, TIME_STEPS)

# Reshape data to (Samples, TimeSteps, Rows, Cols, Channels)
X_train_s1_hybrid = X_train_s1_seq.reshape((-1, TIME_STEPS, IMG_ROWS, IMG_COLS, 1))
X_test_s1_hybrid = X_test_s1_seq.reshape((-1, TIME_STEPS, IMG_ROWS, IMG_COLS, 1))

# Calculate class weights for Stage 1
weights_s1 = class_weight.compute_class_weight(
    'balanced', classes=np.unique(y_train_s1_seq), y=y_train_s1_seq
)
class_weights_dict_s1 = dict(zip(np.unique(y_train_s1_seq), weights_s1))

def build_cnn_lstm_stage1(input_shape):
    model = Sequential()
    # 1. CNN Feature Extractor (wrapped in TimeDistributed)
    model.add(TimeDistributed(
        Conv2D(32, kernel_size=(3, 3), activation='relu'),
        input_shape=input_shape
    ))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Flatten()))
    
    # 2. LSTM Sequence Learner
    model.add(LSTM(50, activation='relu'))
    
    # 3. Classifier Head
    model.add(Dropout(0.5))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    return model

print("\n--- Training Hybrid CNN-LSTM Stage 1 (Detection) ---")
model_s1_hybrid = build_cnn_lstm_stage1(HYBRID_INPUT_SHAPE)
model_s1_hybrid.summary()

model_s1_hybrid.fit(X_train_s1_hybrid, y_train_s1_seq,
                    batch_size=128,
                    epochs=20,
                    validation_split=0.1,
                    class_weight=class_weights_dict_s1,
                    callbacks=[early_stopper])

print("\n--- Hybrid CNN-LSTM Stage 1 Evaluation ---")
y_pred_s1_hybrid = (model_s1_hybrid.predict(X_test_s1_hybrid) > 0.5).astype("int32")
print(classification_report(y_test_s1_seq, y_pred_s1_hybrid, zero_division=0))


# ==========================================================
#  STAGE 2: HYBRID CNN-LSTM (MULTICLASS)
# ==========================================================

print("\n--- Preparing Data for Hybrid Stage 2 ---")
# Create sequences from PADDED data and CATEGORICAL labels
X_train_s2_seq, y_train_s2_seq_cat = create_lstm_dataset(X_train_padded, y_train_s2_cat, TIME_STEPS)
X_test_s2_seq, y_test_s2_seq_cat = create_lstm_dataset(X_test_padded, y_test_s2_cat, TIME_STEPS)

# Reshape X data
X_train_s2_hybrid = X_train_s2_seq.reshape((-1, TIME_STEPS, IMG_ROWS, IMG_COLS, 1))
X_test_s2_hybrid = X_test_s2_seq.reshape((-1, TIME_STEPS, IMG_ROWS, IMG_COLS, 1))

# Create y labels for evaluation
_, y_test_s2_seq_labels = create_lstm_dataset(X_test_padded, y_test_s2_labels.values, TIME_STEPS)

# Calculate class weights for Stage 2
_, y_train_s2_seq_labels = create_lstm_dataset(X_train_padded, y_train_s2_labels.values, TIME_STEPS)
unique_labels_s2_seq = np.unique(y_train_s2_seq_labels)
weights_s2 = class_weight.compute_class_weight(
    'balanced', classes=unique_labels_s2_seq, y=y_train_s2_seq_labels
)
class_weights_dict_s2 = {i: 1.0 for i in range(N_CLASSES)}
calculated_dict_s2 = dict(zip(unique_labels_s2_seq, weights_s2))
class_weights_dict_s2.update(calculated_dict_s2)

def build_cnn_lstm_stage2(input_shape, num_classes):
    model = Sequential()
    # 1. CNN Feature Extractor
    model.add(TimeDistributed(
        Conv2D(32, kernel_size=(3, 3), activation='relu'),
        input_shape=input_shape
    ))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Flatten()))
    
    # 2. LSTM Sequence Learner
    model.add(LSTM(50, activation='relu'))
    
    # 3. Classifier Head
    model.add(Dropout(0.5))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

print("\n--- Training Hybrid CNN-LSTM Stage 2 (Classification) ---")
model_s2_hybrid = build_cnn_lstm_stage2(HYBRID_INPUT_SHAPE, N_CLASSES)
model_s2_hybrid.summary()

model_s2_hybrid.fit(X_train_s2_hybrid, y_train_s2_seq_cat,
                    batch_size=128,
                    epochs=20,
                    validation_split=0.1,
                    class_weight=class_weights_dict_s2,
                    callbacks=[early_stopper])

print("\n--- Hybrid CNN-LSTM Stage 2 Evaluation ---")
y_pred_s2_hybrid_probs = model_s2_hybrid.predict(X_test_s2_hybrid)
y_pred_s2_hybrid_labels = np.argmax(y_pred_s2_hybrid_probs, axis=1)

print(classification_report(y_test_s2_seq_labels, y_pred_s2_hybrid_labels, zero_division=0))

Loading data...
N_CLASSES set to 21
Data prepared with 37 common features.
Scaling and padding data...
Padding from 37 to 49 for (7, 7) image.

--- Preparing Data for Hybrid Stage 1 ---

--- Training Hybrid CNN-LSTM Stage 1 (Detection) ---
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 10, 5, 5, 32)     320       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 10, 2, 2, 32)     0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 10, 128)          0         
 tributed)                                                       
                                                                 
 lstm (LSTM)  

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense, Conv1D, Flatten, MaxPooling1D, Dropout, LSTM
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

# --- HELPER FUNCTIONS ---

def create_lstm_dataset(X, y, time_steps=10):
    """Converts data into sequences for LSTM"""
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps)]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# --- 1. DATA PREPARATION (Load all data) ---
print("Loading data...")
train_data = pd.read_csv('modified_data_train.csv')
test_data  = pd.read_csv('modified_data_test.csv')

# --- Prep for Stage 1 (Binary) ---
train_data['binary_fault'] = train_data['faultNumber'].apply(lambda x: 0 if x == 0 else 1)
test_data['binary_fault'] = test_data['faultNumber'].apply(lambda x: 0 if x == 0 else 1)

y_train_s1 = train_data['binary_fault'].astype(int)
y_test_s1 = test_data['binary_fault'].astype(int)

# --- Prep for Stage 2 (Multiclass) ---
y_train_s2_labels = train_data['faultNumber'].astype(int)
y_test_s2_labels = test_data['faultNumber'].astype(int)

all_labels = pd.concat([y_train_s2_labels, y_test_s2_labels])
N_CLASSES = int(all_labels.max()) + 1
print(f"N_CLASSES set to {N_CLASSES}")

y_train_s2_cat = to_categorical(y_train_s2_labels, num_classes=N_CLASSES)
y_test_s2_cat = to_categorical(y_test_s2_labels, num_classes=N_CLASSES)

# --- Prep Features (X) ---
drop_cols = ['faultNumber', 'simulationRun', 'sample', 'binary_fault']
X_train = train_data.drop(columns=drop_cols, errors='ignore')
X_test = test_data.drop(columns=drop_cols, errors='ignore')

common_cols = sorted(list(set(X_train.columns).intersection(set(X_test.columns))))
X_train = X_train[common_cols]
X_test  = X_test[common_cols]

print(f"Data prepared with {len(common_cols)} common features.")
N_FEATURES = X_train.shape[1] # This will be 37

# --- 2. SCALING (NO PADDING) ---
print("Scaling data...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Define Model Parameters ---
TIME_STEPS = 10
# The input shape is (time_steps, features)
HYBRID_INPUT_SHAPE = (TIME_STEPS, N_FEATURES) 

# Define Early Stopping
early_stopper = EarlyStopping(
    monitor='val_loss', patience=3, verbose=1, restore_best_weights=True
)

# ==========================================================
#  STAGE 1: HYBRID 1D-CNN-LSTM (BINARY)
# ==========================================================

print("\n--- Preparing Data for 1D-CNN-LSTM Stage 1 ---")
# Create sequences from the UNPADDED data
X_train_s1_seq, y_train_s1_seq = create_lstm_dataset(X_train_scaled, y_train_s1.values, TIME_STEPS)
X_test_s1_seq, y_test_s1_seq = create_lstm_dataset(X_test_scaled, y_test_s1.values, TIME_STEPS)

# Calculate class weights for Stage 1
weights_s1 = class_weight.compute_class_weight(
    'balanced', classes=np.unique(y_train_s1_seq), y=y_train_s1_seq
)
class_weights_dict_s1 = dict(zip(np.unique(y_train_s1_seq), weights_s1))

def build_cnn1d_lstm_stage1(input_shape):
    model = Sequential()
    # 1. 1D-CNN Feature Extractor
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    
    # 2. LSTM Sequence Learner
    model.add(LSTM(50, activation='relu'))
    
    # 3. Classifier Head
    model.add(Dropout(0.5))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    return model

print("\n--- Training Hybrid 1D-CNN-LSTM Stage 1 (Detection) ---")
model_s1_hybrid = build_cnn1d_lstm_stage1(HYBRID_INPUT_SHAPE)
model_s1_hybrid.summary()

model_s1_hybrid.fit(X_train_s1_seq, y_train_s1_seq,
                    batch_size=128,
                    epochs=20,
                    validation_split=0.1,
                    class_weight=class_weights_dict_s1,
                    callbacks=[early_stopper])

print("\n--- Hybrid 1D-CNN-LSTM Stage 1 Evaluation ---")
y_pred_s1_hybrid = (model_s1_hybrid.predict(X_test_s1_seq) > 0.5).astype("int32")
print(classification_report(y_test_s1_seq, y_pred_s1_hybrid, zero_division=0))


# ==========================================================
#  STAGE 2: HYBRID 1D-CNN-LSTM (MULTICLASS)
# ==========================================================

print("\n--- Preparing Data for 1D-CNN-LSTM Stage 2 ---")
# Create sequences from UNPADDED data and CATEGORICAL labels
X_train_s2_seq, y_train_s2_seq_cat = create_lstm_dataset(X_train_scaled, y_train_s2_cat, TIME_STEPS)

# --- [FIXED LINE HERE] ---
# Changed 'y_test_cat' to 'y_test_s2_cat'
X_test_s2_seq, y_test_s2_seq_cat = create_lstm_dataset(X_test_scaled, y_test_s2_cat, TIME_STEPS)
# --- [END OF FIX] ---

# Create y labels for evaluation
_, y_test_s2_seq_labels = create_lstm_dataset(X_test_scaled, y_test_s2_labels.values, TIME_STEPS)

# Calculate class weights for Stage 2
_, y_train_s2_seq_labels = create_lstm_dataset(X_train_scaled, y_train_s2_labels.values, TIME_STEPS)
unique_labels_s2_seq = np.unique(y_train_s2_seq_labels)
weights_s2 = class_weight.compute_class_weight(
    'balanced', classes=unique_labels_s2_seq, y=y_train_s2_seq_labels
)
class_weights_dict_s2 = {i: 1.0 for i in range(N_CLASSES)}
calculated_dict_s2 = dict(zip(unique_labels_s2_seq, weights_s2))
class_weights_dict_s2.update(calculated_dict_s2)

def build_cnn1d_lstm_stage2(input_shape, num_classes):
    model = Sequential()
    # 1. 1D-CNN Feature Extractor
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))

    # 2. LSTM Sequence Learner
    model.add(LSTM(50, activation='relu'))
    
    # 3. Classifier Head
    model.add(Dropout(0.5))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

print("\n--- Training Hybrid 1D-CNN-LSTM Stage 2 (Classification) ---")
model_s2_hybrid = build_cnn1d_lstm_stage2(HYBRID_INPUT_SHAPE, N_CLASSES)
model_s2_hybrid.summary()

model_s2_hybrid.fit(X_train_s2_seq, y_train_s2_seq_cat,
                    batch_size=128,
                    epochs=20,
                    validation_split=0.1,
                    class_weight=class_weights_dict_s2,
                    callbacks=[early_stopper])

print("\n--- Hybrid 1D-CNN-LSTM Stage 2 Evaluation ---")
y_pred_s2_hybrid_probs = model_s2_hybrid.predict(X_test_s2_seq)
y_pred_s2_hybrid_labels = np.argmax(y_pred_s2_hybrid_probs, axis=1)

print(classification_report(y_test_s2_seq_labels, y_pred_s2_hybrid_labels, zero_division=0))

Loading data...
N_CLASSES set to 21
Data prepared with 37 common features.
Scaling data...

--- Preparing Data for 1D-CNN-LSTM Stage 1 ---

--- Training Hybrid 1D-CNN-LSTM Stage 1 (Detection) ---
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 8, 64)             7168      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 4, 64)            0         
 1D)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 50)                23000     
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_6 (Dense)             (None, 50)                2

Here we get the result that 1D CNN LSTM model is working better and rather best among all the models built so far.